In [1]:
import tensorflow as tf
from tensorflow import keras

# 构建模型
model = keras.Sequential(
[
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# optimizer：此对象会指定训练过程。从 tf.train 模块向其传递优化器实例，例如 tf.train.AdamOptimizer、tf.train.RMSPropOptimizer 或 tf.train.GradientDescentOptimizer。
# loss：要在优化期间最小化的函数。常见选择包括均方误差 (mse)、categorical_crossentropy 和 binary_crossentropy。损失函数由名称或通过从 tf.keras.losses 模块传递可调用对象来指定。
# metrics：用于监控训练。它们是 tf.keras.metrics 模块中的字符串名称或可调用对象，常见有mae，accuracy

In [2]:
import numpy as np

# 训练模型
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32, validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 217us/step - loss: 11.5246 - acc: 0.0870 - val_loss: 11.7066 - val_acc: 0.0600
Epoch 2/10
1000/1000 [==============================] - 0s 28us/step - loss: 11.4701 - acc: 0.1070 - val_loss: 11.7025 - val_acc: 0.0900
Epoch 3/10
1000/1000 [==============================] - 0s 33us/step - loss: 11.4607 - acc: 0.1040 - val_loss: 11.7021 - val_acc: 0.0900
Epoch 4/10
1000/1000 [==============================] - 0s 44us/step - loss: 11.4528 - acc: 0.1340 - val_loss: 11.6980 - val_acc: 0.0600
Epoch 5/10
1000/1000 [==============================] - 0s 48us/step - loss: 11.4477 - acc: 0.1370 - val_loss: 11.7025 - val_acc: 0.1000
Epoch 6/10
1000/1000 [==============================] - 0s 34us/step - loss: 11.4434 - acc: 0.1500 - val_loss: 11.7050 - val_acc: 0.0500
Epoch 7/10
1000/1000 [==============================] - 0s 41us/step - loss: 11.4387 - acc: 0.1530 - val_loss: 11.7042 - val_acc: 

In [3]:
# 评估模型
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.evaluate(data, labels, batch_size=32)

1000/1000 [==============================] - 0s 15us/step


[11.500132873535156, 0.091]

In [4]:
result = model.predict(data, batch_size=32)
print(result.shape)

(1000, 10)


In [7]:
inputs = keras.Input(shape=(32, ))

x = keras.layers.Dense(64, activation='relu')(inputs)
x = keras.layers.Dense(64, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

In [11]:
model = keras.Model(inputs=inputs, outputs=predictions)

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 212us/step - loss: 11.7386 - acc: 0.0970
Epoch 2/5
1000/1000 [==============================] - 0s 42us/step - loss: 11.5504 - acc: 0.0930
Epoch 3/5
1000/1000 [==============================] - 0s 42us/step - loss: 11.5003 - acc: 0.0840
Epoch 4/5
1000/1000 [==============================] - 0s 38us/step - loss: 11.4903 - acc: 0.1090
Epoch 5/5
1000/1000 [==============================] - 0s 37us/step - loss: 11.4817 - acc: 0.1240


In [12]:
class MyModel(tf.keras.Model):
    
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.dense_1 = keras.layers.Dense(32, activation='relu')
        self.dense_2 = keras.layers.Dense(num_classes, activation='sigmoid')
        
    def call(self, inputs):
        x = self.dense_1(inputs)
        return self.dense_2(x)
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

In [16]:
# 实例化新模型类
model = MyModel(num_classes=10)

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - ETA: 4s - loss: 11.0811 - acc: 0.12 - 0s 187us/step - loss: 11.5214 - acc: 0.1100
Epoch 2/5
1000/1000 [==============================] - 0s 36us/step - loss: 11.5132 - acc: 0.1120
Epoch 3/5
1000/1000 [==============================] - 0s 38us/step - loss: 11.5000 - acc: 0.1270
Epoch 4/5
1000/1000 [==============================] - 0s 35us/step - loss: 11.4928 - acc: 0.1150
Epoch 5/5
1000/1000 [==============================] - 0s 33us/step - loss: 11.4885 - acc: 0.1130


In [18]:
# 使用核矩阵实现输入 matmul 的自定义层
class MyLayer(keras.layers.Layer):
    
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel',
                                      shape=shape,
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)
    
    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config
    
    @classmethod
    def from_config(cld, config):
        return cld(**config)

In [20]:
# 使用自定义层创建模型
model = tf.keras.Sequential([
    MyLayer(10),
    keras.layers.Activation('softmax')
])

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 159us/step - loss: 11.4978 - acc: 0.0790
Epoch 2/5
1000/1000 [==============================] - 0s 35us/step - loss: 11.4897 - acc: 0.0960
Epoch 3/5
1000/1000 [==============================] - 0s 32us/step - loss: 11.4861 - acc: 0.0990
Epoch 4/5
1000/1000 [==============================] - 0s 34us/step - loss: 11.4851 - acc: 0.1070
Epoch 5/5
1000/1000 [==============================] - 0s 31us/step - loss: 11.4822 - acc: 0.0920


In [24]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, labels, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 1s 1ms/step - loss: 11.4762 - acc: 0.1270 - val_loss: 11.6970 - val_acc: 0.0300
Epoch 2/5
1000/1000 [==============================] - 1s 924us/step - loss: 11.4743 - acc: 0.1190 - val_loss: 11.6990 - val_acc: 0.1200
Epoch 3/5
1000/1000 [==============================] - 1s 1ms/step - loss: 11.4727 - acc: 0.1180 - val_loss: 11.7112 - val_acc: 0.0600


In [26]:
# 保存和恢复
# tf.keras.Model.save_weights 保存并加载模型的权重
model = tf.keras.Sequential([
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
             metrics=['accuracy'])

model.save_weights('./weights/my_model')

model.load_weights('./weights/my_model')

In [27]:
# 保存模型的配置，此操作会对模型架构（不含任何权重）进行序列化Keras 支持 JSON 和 YAML 序列化格式
json_string = model.to_json()
json_string

'{"class_name": "Sequential", "config": {"name": "sequential_2", "layers": [{"class_name": "Dense", "config": {"name": "dense_16", "trainable": true, "dtype": null, "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_17", "trainable": true, "dtype": null, "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}]}, "ke

In [28]:
import json
import pprint
pprint.pprint(json.loads(json_string))

{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'layers': [{'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {'dtype': 'float32'}},
                                   'bias_regularizer': None,
                                   'dtype': None,
                                   'kernel_constraint': None,
                                   'kernel_initializer': {'class_name': 'GlorotUniform',
                                                          'config': {'dtype': 'float32',
                                                                     'seed': None}},
                                   'kernel_regularizer': None,
                                   'name': 'dense

In [29]:
# 从 json 重新创建模型（刚刚初始化）
fresh_model = tf.keras.models.model_from_json(json_string)
yaml_string = model.to_yaml()
print(yaml_string)

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {dtype: float32}
      bias_regularizer: null
      dtype: null
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config: {dtype: float32, seed: null}
      kernel_regularizer: null
      name: dense_16
      trainable: true
      units: 64
      use_bias: true
  - class_name: Dense
    config:
      activation: softmax
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {dtype: float32}
      bias_regularizer: null
      dtype: null
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config: {dtype: float32, seed: null}
      kernel_regularizer: null
      name: dense_17
      t

In [30]:
# 子类化模型不可序列化，因为它们的架构由 call 方法正文中的 Python 代码定义。
# 从 yaml 重新创建模型
fresh_model = tf.keras.models.model_from_yaml(yaml_string)

In [31]:
# 整个模型可以保存到一个文件中，其中包含权重值、模型配置乃至优化器配置。这样，您就可以对模型设置检查点并稍后从完全相同的状态继续训练，而无需访问原始代码
model = tf.keras.Sequential([
    keras.layers.Dense(10, activation='softmax', input_shape=(32,)),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=5)

model.save('my_model.h5')

model = tf.keras.models.load_model('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 260us/step - loss: 11.4854 - acc: 0.1050
Epoch 2/5
1000/1000 [==============================] - 0s 39us/step - loss: 11.4825 - acc: 0.1100
Epoch 3/5
1000/1000 [==============================] - 0s 40us/step - loss: 11.4819 - acc: 0.1100
Epoch 4/5
1000/1000 [==============================] - 0s 35us/step - loss: 11.4813 - acc: 0.1120
Epoch 5/5
1000/1000 [==============================] - 0s 37us/step - loss: 11.4809 - acc: 0.1230


In [3]:
model = tf.keras.Sequential([
    keras.layers.Dense(10, activation='softmax'),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
             metrics=['accuracy'])
estimator = tf.keras.estimator.model_to_estimator(model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\lxg\\AppData\\Local\\Temp\\tmpeml67p_3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F07084E0F0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
